# Weighted constraints
## Exploring the exact conditions for optimal feedforward

The previous notebook demonstrated that the feedforward for changing weights is properly generated by eTaSL.  This notebook shows some cases where the feedforward i still optimally generated, but the condition $\frac{\partial^2 e}{\partial q_i \partial t} = 0$ is violated.
This is caused by $   J_i = J_j,   \forall i, j $ (for all conflicting constraints).  In these cases optimal feedforward is still generated.

In [ ]:
%matplotlib inline
import numpy as np
from etasl_py.etasl import etasl_simulator,array_to_dict,dict_to_array,to_deg, to_rad
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from etasl_py.bokehplots import plotv, append_units

output_notebook()

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; } </style>"))

## Constraints expressed in the same point/frame

i.e. $J_i = J_j ~~~\forall i,j$, so we expect that optimal feedforward is generated.

We use the same example as in [weights_1](weights_1.ipynb), but we put the robot on a turn-table that is moving along a fixed sine wave trajectory.  In this case  $\frac{\partial^2 e}{\partial q_i \partial t} \neq 0$.

In [ ]:
etaslspec3="""
    require("context")
    require("geometric")
    -- Robot:
    local u=UrdfExpr();
    local fn = rospack_find("etasl_py_examples").."/robots/ur10_robot.urdf"
    u:readFromFile(fn)
    u:addTransform("ee","tool0","base_link")
    local r = u:getExpressions(ctx)
    ee = r.ee
    robot_jname={"shoulder_pan_joint","shoulder_lift_joint","elbow_joint","wrist_1_joint","wrist_2_joint","wrist_3_joint"}
    robot_jval = {}
    for i=1,#robot_jname do
       robot_jval[i]   = ctx:getScalarExpr(robot_jname[i])
    end
    -- robot on a turn-table rotating around z with angle = 0.3*sin(1.5*time)
    ee = rotate_z( 0.3*sin(1.5*time))*ee
    -- Task:
    control_gain = ctx:createInputChannelScalar("control_gain",0)
    tgt_x  = sin(time)*0.15 + 0.7
    tgt_z  = 0
    tgt1_y = sin(2.5*time)*0.1 +0.4
    tgt2_y = 0
    w1 = 1-sin(0.5*time)*0.8
    w2 = 1+cos(0.5*time)*0.9
    d = Variable{context=ctx, name="d", vartype="feature"}

    Constraint{
        context=ctx,
        name="laserdistance",
        expr=d,
        target_lower = 0.3,
        target_upper = 0.45,
        K = control_gain
    }

    laserspot = ee*vector(0,0,d)

    Constraint{
        context=ctx,
        name="x",
        expr = tgt_x - coord_x(laserspot),
        priority = 2,
        K        = control_gain
    }
    Constraint{
        context=ctx,
        name="z",
        expr = tgt_z - coord_z(laserspot),
        priority = 2,
        K        = control_gain
    }
    
    -- conflicting constraints : 
    Constraint{
        context=ctx,
        name="y",
        expr = (tgt1_y - coord_y(laserspot)),
        priority = 2,
        K        = control_gain,
        weight   = w1
    }
    Constraint{
        context=ctx,
        name="y",
        expr = (tgt2_y - coord_y(laserspot)),
        priority = 2,
        K        = control_gain,
        weight   = w2
    }    
    -- Output:
    ctx:setOutputExpression("error_x",coord_x(laserspot)-tgt_x)
    ctx:setOutputExpression("error_y1",coord_y(laserspot)-tgt1_y)
    ctx:setOutputExpression("error_y2",coord_y(laserspot)-tgt2_y)
    ctx:setOutputExpression("error_y_weighted",w1*(coord_y(laserspot)-tgt1_y) + w2*(coord_y(laserspot)-tgt2_y) ) 
    ctx:setOutputExpression("error_z",coord_z(laserspot)-tgt_z)    
"""
e3 = etasl_simulator(regularization_factor= 0.0000001)
e3.readTaskSpecificationString(etaslspec3)

In [ ]:
pos_lbl = ['shoulder_pan_joint','shoulder_lift_joint', 'elbow_joint', 
           'wrist_1_joint', 'wrist_2_joint',  'wrist_3_joint','d']
initial_jpos = np.array([0, -np.pi*0.6 , np.pi*0.6,-2.0,-np.pi*0.5,0.1,0.1])
N=4000
dt=0.005


inp_lbl=['control_gain']
inp=[4]
e3.setInputTable(inp_lbl,inp)

e3.initialize(initial_jpos, pos_lbl)
e3.simulate(N=N,dt=dt)

After the simulation, we present the results using Bokeh:

In [ ]:
f = figure(x_axis_label='time [s]', 
           y_axis_label= 'weighted error [m]',plot_width=600, plot_height=300,
           title='weighted error',tooltips="$x[s] : $y[m]" )
f.line(e3.TIME,e3.OUTP[:,3], line_width=2)
show(f)
f = figure(x_axis_label='time [s]', 
                y_axis_label= 'weighted error [m]',plot_width=600, plot_height=300, y_range=(-0.002,0.002),
                title='weighted error (zoomed in)',tooltips="$x[s] : $y[m]")
f.line(e3.TIME,e3.OUTP[:,3], line_width=2)
show(f)

An now we see that the weighted error evolves towards 0.  If we zoom in there is a remaining tracking error of $1.33 mm$.  Could this remaining error be because of linearization errors? Let's check this by rerunning the simulaton with a **smaller sample period**.

In [ ]:
e4 = etasl_simulator(regularization_factor= 0.0000001)
e4.readTaskSpecificationString(etaslspec3)
inp_lbl=['control_gain']
inp=[4]
e4.setInputTable(inp_lbl,inp)
e4.initialize(initial_jpos, pos_lbl)
e4.simulate(N=20000,dt=0.001)

In [ ]:
f = figure(x_axis_label='x[m]', 
                y_axis_label= 'y[m]',plot_width=600, plot_height=300, y_range=(-0.002,0.002),
                title='weighted error (zoomed in)',tooltips="$x[s] : $y[m]")
f.line(e4.TIME,e4.OUTP[:,3], line_width=2)
show(f)

The tracking error is now reduced ( to $0.27 mm$).  This shows that the remaining tracking error is indeed a **linearization error**: the tracking error reduces when sample period is reduced.  
